# 📊 Analyse Interactive du Dataset Iris

## Objectif
Ce notebook effectue une analyse complète du dataset Iris en utilisant pandas, matplotlib et seaborn. Nous allons explorer, nettoyer, analyser et visualiser les données étape par étape.

**Dataset** : Iris Dataset (150 observations de 3 espèces d'iris)
**Variables** : 4 mesures florales + 1 classification d'espèce

## Section 1 : Charger et Inspecter le Dataset

Nous allons charger le dataset Iris en utilisant sklearn et pandas, puis afficher les premières lignes.

In [ ]:
# Importer les bibliothèques nécessaires
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.datasets import load_iris
from datetime import datetime, timedelta
import warnings

warnings.filterwarnings('ignore')

# Configuration pour les visualisations
sns.set_style("whitegrid")
sns.set_palette("husl")

print("✓ Toutes les bibliothèques ont été importées avec succès!")

In [ ]:
# Charger le dataset Iris avec gestion des erreurs
try:
    # Charger les données
    iris = load_iris()
    df = pd.DataFrame(iris.data, columns=iris.feature_names)
    df['species'] = iris.target_names[iris.target]
    
    # Ajouter une colonne de date pour la visualisation temporelle
    dates = pd.date_range(start='2023-01-01', periods=len(df), freq='2D')
    df['date'] = dates
    
    print(f"✓ Dataset chargé avec succès!")
    print(f"  • Nombre de lignes: {df.shape[0]}")
    print(f"  • Nombre de colonnes: {df.shape[1]}")
    
except Exception as e:
    print(f"✗ Erreur lors du chargement: {e}")

In [ ]:
# Afficher les premières lignes du dataset
print("\n" + "="*80)
print("PREMIÈRES LIGNES DU DATASET (.head())")
print("="*80 + "\n")
print(df.head(10))
print(f"\n... Total de {len(df)} lignes disponibles ...")

## Section 2 : Explorer la Structure des Données

Vérifions les types, la forme du dataset et recherchons des valeurs manquantes.

In [ ]:
# Types de données et information générale
print("Types de données (.dtypes):")
print(df.dtypes)

print("\nRésumé du dataset (.info()):")
print(df.info())

# Valeurs manquantes
print("\nValeurs manquantes par colonne:")
print(df.isnull().sum())

# Pourcentage des valeurs manquantes
missing_percent = (df.isnull().sum() / len(df)) * 100
print("\nPourcentage des valeurs manquantes:")
print(missing_percent)

## Section 3 : Nettoyer le Dataset

Stratégies :
- Supprimer les doublons
- Remplir les valeurs manquantes (si présentes) avec la médiane pour les colonnes numériques
- Vérifier à nouveau la qualité des données

In [ ]:
# Nettoyage des données
print("Nombre de doublons avant nettoyage:", df.duplicated().sum())
df_clean = df.drop_duplicates()
print("Nombre de doublons après nettoyage:", df_clean.duplicated().sum())

# Remplir les valeurs manquantes si nécessaire
if df_clean.isnull().sum().sum() > 0:
    numeric_cols = df_clean.select_dtypes(include=[np.number]).columns
    df_clean[numeric_cols] = df_clean[numeric_cols].fillna(df_clean[numeric_cols].median())
    print("Valeurs manquantes remplies avec la médiane")
else:
    print("Aucune valeur manquante détectée")

# Vérification finale
print("\nDataset après nettoyage:")
print(df_clean.shape)
print(df_clean.isnull().sum())

## Section 4 : Calculer les Statistiques Descriptives

Utilisation de `.describe()` pour obtenir les statistiques de base et interprétation rapide.

In [ ]:
# Statistiques descriptives
stats = df_clean.describe()
print(stats)

# Calculs additionnels (médiane, variance)
print("\nMédianes:")
print(df_clean.median(numeric_only=True))

print("\nÉcarts-types:")
print(df_clean.std(numeric_only=True))

## Section 5 : Regroupements et Analyses

Nous allons grouper par `species` et calculer les moyennes des colonnes numériques.

In [ ]:
# Regroupement par espèce
species_group = df_clean.groupby('species')
print("Moyennes par espèce:")
print(species_group.mean())

# Comptage par espèce
print("\nNombre par espèce:")
print(df_clean['species'].value_counts())

# Statistiques supplémentaires par espèce
print("\nStatistiques (mean, std) par espèce pour les colonnes numériques:")
print(species_group.agg(['mean', 'std']))

# Observations rapides
print("\nObservations rapides:")
for s in df_clean['species'].unique():
    mean_petal = df_clean[df_clean['species'] == s]['petal length (cm)'].mean()
    print(f" - {s}: longueur moyenne du pétale = {mean_petal:.2f} cm")

## Section 6 : Graphique Linéaire (Tendance Temporelle)

Nous traçons la moyenne quotidienne de la longueur des sépales pour visualiser une tendance temporelle (les dates sont simulées).

In [ ]:
# Graphique linéaire - moyenne quotidienne de la longueur des sépales
plt.figure(figsize=(12, 5))
daily_avg = df_clean.groupby('date')['sepal length (cm)'].mean()
plt.plot(daily_avg.index, daily_avg.values, marker='o', linestyle='-', color='#1f77b4')
plt.fill_between(daily_avg.index, daily_avg.values, alpha=0.2, color='#1f77b4')
plt.title('Tendance temporelle de la longueur des sépales')
plt.xlabel('Date')
plt.ylabel('Longueur des sépales (cm)')
plt.xticks(rotation=45)
plt.grid(alpha=0.3)
plt.show()

## Section 7 : Graphique en Barres

Comparaison de la longueur moyenne des pétales par espèce.

In [ ]:
# Graphique en barres - longueur moyenne des pétales par espèce
plt.figure(figsize=(8, 5))
species_petal = df_clean.groupby('species')['petal length (cm)'].mean()
sns.barplot(x=species_petal.index, y=species_petal.values, palette='pastel')
plt.title('Longueur moyenne des pétales par espèce')
plt.xlabel('Espèce')
plt.ylabel('Longueur des pétales (cm)')
for i, v in enumerate(species_petal.values):
    plt.text(i, v + 0.05, f"{v:.2f}", ha='center', fontweight='bold')
plt.show()

## Section 8 : Histogramme

Distribution d'une variable numérique (longueur des sépales).

In [ ]:
# Histogramme - distribution de la longueur des sépales
plt.figure(figsize=(8, 5))
plt.hist(df_clean['sepal length (cm)'], bins=20, color='#2ca02c', edgecolor='black', alpha=0.7)
plt.title('Histogramme : Longueur des Sépales')
plt.xlabel('Longueur des Sépales (cm)')
plt.ylabel('Fréquence')
plt.axvline(df_clean['sepal length (cm)'].mean(), color='red', linestyle='--', label=f"Moyenne: {df_clean['sepal length (cm)'].mean():.2f}")
plt.legend()
plt.show()

## Section 9 : Nuage de Points (Scatter Plot)

Visualiser la relation entre la longueur des sépales et la longueur des pétales.

In [ ]:
# Scatter plot - sepal length vs petal length colored by species
plt.figure(figsize=(8, 6))
palette = {'setosa': '#1f77b4', 'versicolor': '#ff7f0e', 'virginica': '#2ca02c'}
for s in df_clean['species'].unique():
    subset = df_clean[df_clean['species'] == s]
    plt.scatter(subset['sepal length (cm)'], subset['petal length (cm)'], label=s, 
                s=80, alpha=0.7, edgecolor='black')

# Ligne de tendance générale
z = np.polyfit(df_clean['sepal length (cm)'], df_clean['petal length (cm)'], 1)
p = np.poly1d(z)
x_vals = np.linspace(df_clean['sepal length (cm)'].min(), df_clean['sepal length (cm)'].max(), 100)
plt.plot(x_vals, p(x_vals), "k--", linewidth=1.8, label='Tendance')

plt.title('Relation: Longueur Sépale vs Longueur Pétale')
plt.xlabel('Longueur Sépale (cm)')
plt.ylabel('Longueur Pétale (cm)')
plt.legend()
plt.grid(alpha=0.3)
plt.show()

## Fin et Conclusion

Résumé des étapes réalisées et suggestions pour aller plus loin (modélisation, PCA, clustering).